In [35]:
import json
import pandas as pd 
import sys
from collections import defaultdict
import numpy as np

sys.path.append('..')
import ENVIRONMENT_VARIABLES as EV

In [36]:
years_analysed = [2023]
teams = ["Broncos", "Roosters", "Wests Tigers", "Rabbitohs", "Storm", "Eels", "Raiders", "Knights", "Dragons", "Sea Eagles", "Panthers", "Sharks", "Bulldogs", "Dolphins", "Titans", "Cowboys", "Warriors"]
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round"]
years =  [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2022, 2023]

# Match Data

In [37]:
teams = EV.TEAMS
variables =["Year", "Win", "Defense", "Attack", "Margin", "Home", "Versus",  "Round", "Halftime_Attack", "Halftime_Defense"]
years =  [2023]

# Create a DataFrame with columns representing combinations of team and variable names
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in variables])

In [38]:

# Initialize an empty dictionary to store data for each year
years_arr = {}

# Open the JSON file containing NRL data
with open('./data/nrl_data_2023.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    
    # Extract NRL data from the loaded JSON
    data = data['NRL']
    
    # Iterate over each year in the years list
    for year in years:
        # Extract data for the current year and store it in the years_arr dictionary
        # Note: years.index(year) returns the index of the current year in the years list
        #       This index is then used to access the corresponding data for that year
        years_arr[year] = data[years.index(year)][str(year)]


In [46]:
round_game_data = defaultdict(dict)

with open(f'./data/nrl_detailed_match_data_2023.json', 'r') as file:
    # Load JSON data from the file
    data = json.load(file)
    data = data['NRL']
    for round in range(0, 25):
        round_data = data[round]
        for key, value in round_data.items():
            game_datas = round_data[key]
            for game in game_datas:
                # Now the game 
                game_name = list(game.keys())[0]
                round_game_data[round][game_name] = game[game_name]

In [47]:

# Initialize an empty list to store data for all rounds
all_store = []

# Iterate over each year in the years list
for year in years:
    # Iterate over each round (assuming 26 rounds)
    for round in range(0, 25):
        try:
            # Extract data for the current round
            round_data = years_arr[year][round][str(round+1)]
            
            # Create an empty feature array 
            round_store = np.zeros([len(teams)*len(variables)], dtype=int)
            round_teams = []
            
            # Iterate over each game in the round data
            for game in round_data:
                # Extract information about the game
                h_team = game['Home']
                h_score = int(game['Home_Score'])
                a_team = game['Away']
                a_score = int(game['Away_Score'])
                
                h_team_name = h_team.replace(" ", "-")
                a_team_name = a_team.replace(" ", "-")
                

                rgd = round_game_data[round][f"{h_team} v {a_team}"]
                match_data_away = rgd['away']['half_time']
                match_data_home = rgd['home']['half_time']
                

                print(f"{round} {h_team_name} {a_team_name} {match_data_away}")
                


                
                # home_team_data = rgd['home']
                # home_penalitites = home_team_data['penalties_conceded']
                
                
                # away_team_data = rgd['away']
                # away_penalitites = away_team_data['penalties_conceded']
                
                
                # Determine win or lose for each team
                h_team_win, a_team_win = h_score >= a_score, a_score >= h_score
                
                # Determine home team status
                h_home, a_home = 1, 0
                
                # Determine versus index
                h_versus, a_versus= teams.index(a_team), teams.index(h_team)
                
                # Determine defense (points let in)
                h_team_defense = a_score
                a_team_defense = h_score  
                
                # Determine attack points scored
                h_team_attack = h_score 
                a_team_attack = a_score   
                
                # Determine margin
                h_team_margin =  h_score - a_score   
                a_team_margin =  a_score - h_score        
                
                # Keep track of which teams played to work out which teams had a bye 
                round_teams.append(h_team)
                round_teams.append(a_team)
                
                # Find the index of the team in the overarching array 
                a_team_idx = teams.index(a_team)
                h_team_idx = teams.index(h_team)
                
                # Determine feature map index
                a_team_idx_fm = a_team_idx * len(variables)
                h_team_idx_fm = h_team_idx * len(variables)
                
                # Populate data for away team
                for idx, data in zip(range(a_team_idx_fm, a_team_idx_fm + 10), 
                                    [year, a_team_win, a_team_defense, a_team_attack, a_team_margin, a_home, a_versus, round + 1, match_data_away, match_data_home]):
                    round_store[idx] = data

                # Populate data for home team
                for idx, data in zip(range(h_team_idx_fm, h_team_idx_fm + 10),
                                    [year, h_team_win, h_team_defense, h_team_attack, h_team_margin, h_home, h_versus, round + 1, match_data_home, match_data_away]):
                    round_store[idx] = data
                
            # Determine teams with a bye
            bye_teams = list(set(teams) - set(round_teams))
            
            # Assign values for teams with a bye
            for bye_team in bye_teams:
                b_team_idx = teams.index(bye_team)
                b_team_idx_fm = b_team_idx * len(variables)
                round_store[b_team_idx_fm] = year
                round_store[b_team_idx_fm+1] = -1
                round_store[b_team_idx_fm+2] = -1
                round_store[b_team_idx_fm+3] = -1
                round_store[b_team_idx_fm+4] = 0
                round_store[b_team_idx_fm+5] = -1
                round_store[b_team_idx_fm+6] = -1
                round_store[b_team_idx_fm+7] = round+1
                round_store[b_team_idx_fm+8] = -1
                
                
            # Append the round data to the all_store list
            all_store.append(round_store)
            
            # Add the new row to the DataFrame using loc
            df.loc[len(df)] = round_store
        except Exception as ex:
            print(ex)
        

0 Eels Storm 0/2
invalid literal for int() with base 10: '0/2'
1 Panthers Rabbitohs 0
1 Eels Sharks 18
1 Broncos Cowboys 0
invalid literal for int() with base 10: '0/1'
'Sea Eagles v Eels'
'Eels v Panthers'
'Roosters v Eels'
'Storm v Roosters'
'Dolphins v Rabbitohs'
'Rabbitohs v Panthers'
'Sharks v Cowboys'
'Bulldogs v Raiders'
'Storm v Broncos'
'Broncos v Panthers'
'Dolphins v Dragons'
'Wests Tigers v Raiders'
'Titans v Wests Tigers'
'Cowboys v Panthers'
'Dragons v Warriors'
'Sharks v Dragons'
'Wests Tigers v Sharks'
'Knights v Wests Tigers'
'Dragons v Wests Tigers'
'Broncos v Roosters'
'Roosters v Sea Eagles'
'Sea Eagles v Panthers'
'Cowboys v Sharks'


In [48]:
filtered_columns = [col for col in df.columns if 'halftime_attack' in col.lower() or 'halftime_defense' in col.lower()or 'attack' in col.lower() or 'defense' in col.lower()]
filtered_df = df[filtered_columns]

In [49]:
for team in EV.TEAMS:
    # Calculate the difference between Attack and Halftime columns
    attack_col = f"{team} Attack"
    attack_def_col = f"{team} Defense"
    halftime_col = f"{team} Halftime_Attack"
    halftime_def_col = f"{team} Halftime_Defense"
    second_half_col = f"{team} SecondHalf_Attack"
    second_half_def_col = f"{team} SecondHalf_Attack"
    filtered_df[second_half_col] = filtered_df[attack_col] - filtered_df[halftime_col]
    filtered_df[second_half_def_col] = filtered_df[attack_def_col] - filtered_df[halftime_def_col]

filtered_df = filtered_df.reindex(sorted(filtered_df.columns), axis=1)


In [ ]:
filtered_df

,Broncos Attack,Broncos Defense,Broncos Halftime_Attack,Broncos Halftime_Defense,Broncos SecondHalf_Attack,Bulldogs Attack,Bulldogs Defense,Bulldogs Halftime_Attack,Bulldogs Halftime_Defense,Bulldogs SecondHalf_Attack,...,Warriors Attack,Warriors Defense,Warriors Halftime_Attack,Warriors Halftime_Defense,Warriors SecondHalf_Attack,Wests Tigers Attack,Wests Tigers Defense,Wests Tigers Halftime_Attack,Wests Tigers Halftime_Defense,Wests Tigers SecondHalf_Attack
0,13,12,12,6,6,6,31,6,12,19,...,20,12,10,6,6,10,22,2,12,10
1,28,16,10,10,6,26,12,16,0,12,...,12,20,6,8,12,12,14,0,1,13
2,40,18,8,16,2,26,22,20,6,16,...,26,12,16,12,0,22,26,6,20,6
3,18,12,4,0,12,14,16,8,6,10,...,16,14,6,8,6,12,24,6,22,2
4,46,12,28,0,12,15,14,6,8,6,...,32,30,12,26,4,12,46,0,28,18
5,14,20,4,14,6,16,50,6,26,24,...,24,34,6,22,12,22,28,6,18,10
6,43,26,10,14,12,4,30,0,16,14,...,22,14,10,8,6,-1,-1,-1,0,-1
7,26,16,22,6,10,20,33,14,18,15,...,22,30,18,12,18,16,22,6,12,10
8,6,32,6,12,20,18,16,10,4,12,...,0,14,0,8,6,12,8,8,8,0
9,32,6,20,0,6,30,34,18,22,12,...,6,18,6,12,6,18,16,14,10,6


# Player Data

In [50]:
player_variables =["Name"] +EV.PLAYER_LABELS

In [51]:
# Initialize an empty dictionary to store player statistics for each year
years_arr = {}

# Open the JSON file containing player statistics data for a specific year
with open('./data/player_statistics_2023.json', 'r') as file:
    data = json.load(file)  # Load JSON data from the file
    data = data['PlayerStats']  # Extract the 'PlayerStats' key from the JSON data
    for year in years_analysed:  # Iterate over the years to be analyzed
        # Store player statistics data for the current year in the 'years_arr' dictionary
        years_arr[year] = data[years_analysed.index(year)][str(year)]

In [52]:
df = pd.DataFrame(columns=[f"{team} {variable}" for team in teams for variable in player_variables])

In [ ]:
df

,Broncos Year,Broncos Win,Broncos Defense,Broncos Attack,Broncos Margin,Broncos Home,Broncos Versus,Broncos Round,Broncos Halftime_Attack,Broncos Halftime_Defense,...,Warriors Year,Warriors Win,Warriors Defense,Warriors Attack,Warriors Margin,Warriors Home,Warriors Versus,Warriors Round,Warriors Halftime_Attack,Warriors Halftime_Defense


In [53]:

player_stats = defaultdict(list)  # Dictionary to store player statistics

for year in years_analysed:  # Loop through each year to analyze
    for i in range(0, 25):  # Loop through each round (assuming there are 26 rounds)
        try:
            round = years_arr[year][i]  # Get data for a particular round
            round = round[str(i)]  # Assuming round is stored as a string key
            for round_game in round:  # Loop through each game in the round
                for game in round_game:  # Loop through each team's performance in the game
                    # Splitting game data to extract year, round, home team, and away team
                    game_split = game.split("-")
                    game_year, game_round = game_split[0], game_split[1]

                    game_split = game.split("v")
                    home_team = " ".join(game_split[0].split("-")[2:]).replace("-", " ").strip()
                    away_team = " ".join(game_split[-1:]).replace("-", " ").strip()

                    players = round_game[game]  # List of players in the game
                    player_round_stats = {}  # Dictionary to store player statistics for the game
                    for player in players:  # Loop through each player's stats
                        vals = [player[val] for val in player_variables]  # Extract required stats for each player
                        player_round_stats[vals[0]] = vals[1:]  # Store player stats with player ID as key
                    
                    player_round_stats = list(player_round_stats.items())  # Convert player stats to list of tuples
                    # Split player stats into home and away team stats
                    player_round_stats_home, player_round_stats_away = player_round_stats[:18], player_round_stats[18:]
                    
                    # Append home team player stats to player_stats dictionary
                    for player in player_round_stats_home:
                        player_stats[player[0]].append([player[1], game_round, home_team, away_team])
                    # Append away team player stats to player_stats dictionary
                    for player in player_round_stats_away:
                        player_stats[player[0]].append([player[1], game_round, away_team, home_team])
        except IndexError as ex:
            print(f"Run out of round data, round: {i}, \n{ex}")
                    
        except Exception as ex:
            print(f"{ex}\tround: {i}:{game}")  # Print any exceptions that occur during processing


In [54]:
# Combine player labels with additional information
headers = EV.PLAYER_LABELS + ["Round", "Team", "Opposition"]
players = {}

# Loop through each player and their statistics
for player, values in player_stats.items():
    print(player)
    # Combine round statistics with additional information
    player_stat = [[*round[0], *round[1:]] for round in values]
    # Create a DataFrame from the combined statistics and headers
    df = pd.DataFrame(player_stat, columns=headers)
    players[player] = df

Clinton Gutherson
Maika Sivo
Will Penisini
Waqa Blake
Dylan Brown
Mitchell Moses
Reagan Campbell-Gillard
Josh Hodgson
Junior Paulo
Bryce Cartwright
Matt Doorey
J'maine Hopgood
Jirah Momoisea
Jack Murchie
Wiremu Greig
Makahesi Makatoa
Jake Arthur
Isaac Lumelume
Nick Meaney
Will Warbrick
Reimis Smith
Young Tonumaipea
Xavier Coates
Cameron Munster
Jahrome Hughes
Nelson Asofa-Solomona
Harry Grant
Christian Welch
Trent Loiero
Eliesa Katoa
Josh King
Tyran Wishart
Alec MacDonald
Chris Lewis
Jordan Grant
Bronson Garlick
Charnze Nicoll-Klokstad
Brayden Wiliame
Adam Pompey
Marcelo Montoya
Te Maire Martin
Shaun Johnson
Addin Fonua-Blake
Wayde Egan
Mitchell Barnett
Jackson Ford
Marata Niukore
Tohu Harris
Dylan Walker
Bunty Afoa
Josh Curran
Tom Ale
Freddy Lussick
Edward Kosi
Lachlan Miller
Hymel Hunt
Enari Tuala
Bradman Best
Dominic Young
Kalyn Ponga
Jackson Hastings
Daniel Saifiti
Jayden Brailey
Jacob Saifiti
Tyson Frizell
Lachlan Fitzgibbon
Kurt Mann
Phoenix Crossland
Adam Elliott
Jack Hetheringt

In [55]:
players['Clinton Gutherson']

,Number,Position,Mins Played,Points,Tries,Conversions,Conversion Attempts,Penalty Goals,Goal Conversion Rate,1 Point Field Goals,...,Ruck Infringements,Inside 10 Metres,On Report,Sin Bins,Send Offs,Stint One,Stint Two,Round,Team,Opposition
0,1,Fullback,84:00,-,-,-,-,-,-,-,...,-,-,-,-,-,84:00,-,1,Eels,Storm
1,1,Fullback,65:00,-,-,-,-,-,-,-,...,-,1,-,-,-,07:32,57:39,2,Eels,Sharks
2,1,Fullback,80:00,8,2,-,-,-,-,-,...,-,-,-,-,-,80:00,-,3,Eels,Sea Eagles
3,1,Fullback,83:00,-,-,-,-,-,-,-,...,-,-,-,-,-,83:00,-,4,Eels,Panthers
4,1,Fullback,80:00,4,1,-,-,-,-,-,...,1,-,-,-,-,80:00,-,5,Eels,Roosters
5,1,Fullback,80:00,4,1,-,-,-,-,-,...,-,-,-,-,-,80:00,-,6,Eels,Wests Tigers
6,1,Fullback,80:00,-,-,-,-,-,-,-,...,-,-,-,-,-,80:00,-,7,Eels,Bulldogs
7,1,Fullback,80:00,4,1,-,-,-,-,-,...,-,-,-,-,-,80:00,-,8,Eels,Broncos
8,1,Fullback,80:00,12,3,-,-,-,-,-,...,-,-,-,-,-,80:00,-,9,Eels,Knights
9,1,Fullback,70:00,6,1,1,1,-,100%,-,...,-,-,-,1,-,70:00,-,10,Eels,Titans


In [ ]:

# team_1_name = "Dragons"
# team_1 =['Tyrell Sloan', 'Zac Lomax', 'Mosese Suli', 'Jack Bird', 'Mikaele Ravalawa', 'Kyle Flanagan',
#        'Ben Hunt', 'Jack De Belin', 'Jesse Marschke',
#        'Blake Lawrie', 'Luciano Leilua', 'Raymond Faitala-Mariner', 'Tom Eisenhuth']

# team_2_name = "Cowboys"
# team_2 =['Scott Drinkwater', 'Kyle Feldt', 'Valentine Holmes', 'Zac Laybutt', 'Murray Taulagi', 
#          'Tom Dearden', 'Chad Townsend', 'Jordan McLean', 'Reece Robson', 'Jason Taumalolo',
#          'Kulikefu Finefeuiaki', 'Jeremiah Nanai', 'Reuben Cotter']

# team_1_name = "Bulldogs"
# team_1 =['Blake Taaffe', 'Blake Wilson','Jacob Kiraz', 'Stephen Crichton', 'Josh Addo-Carr', 'Matt Burton',
#          'Drew Hutchison', 'Max King', 'Reed Mahoney', 'Liam Knight', 'Villame Kikau',
#          'Jacob Preston', 'Jaeman Salmon']

# team_2_name = "Titans"
# team_2 =['Keano Kini', 'Alofiana Khan-Pereira', 'Brian Kelly', 'AJ Brimson', 'Phillip Sami',
#          'Kieran Foran', 'Tanah Boyd', 'Moeaki Fotuaika', 'Chris Randall', 'Tino Fa\'asuamaleaui', 'Klese Haas',
#          'Beau Fermor', 'Jaimin Jolliffe']


# team_1_name = "Warriors"
# team_1 =['Taine Tuaupiki', 'Dallin Watene-Zelezniak', 'Rocco Berry', 'Roger Tuivasa-Sheck', 'Marcelo Montoya',
#          'Luke Metcalf', 'Shaun Johnson', 'Addin Fonua-Blake', 'Freddy Lussick', 'Mitchell Barnett',
#          'Jackson Ford', 'Kurt Capewell', 'Tohu Harris']

# team_2_name = "Raiders"
# team_2 =['Jordan Rapana' ,'Nick Cotric', 'Matthew Timoko', 'Albert Hopoate', 'Xavier Savage', 'Ethan Strange',
#          'Jamal Fogarty', 'Josh Papali\'i', 'Danny Levi', 'Joseph Tapine', 'Hudson Young', 'Elliott Whitehead',
#          'Morgan Smithies']

# team_1_name = "Wests Tigers"
# team_1 = ['Jahream Bula', 'Charlie Staines', 'Solomona Faataape', 'Justin Olam', 'Junior Tupou', 
#           'Lachlan Galvin', 'Aidan Sezer', 'Stefano Utoikamanu', 'Apisai Koroisau', 'David Klemmer', 
#           'Isaiah Papali\'i', 'John Bateman', 'Fonua Pole']
# team_2_name = "Sharks"
# team_2 = ['William Kennedy', 'Sione Katoa', 'Jesse Ramien' ,'Siosifa Talakai', 'Ronaldo Mulitalo', 'Braydon Trindall',
#           'Nicholas Hynes', 'Toby Rudolf', 'Blayke Brailey', 'Oregon Kaufusi', 'Jack Williams',
#           'Teig Wilton', 'Cameron McInnes']


team_1_name = "Eels"
team_1 = ['Clinton Gutherson', 'Morgan Harper', 'Will Penisini', 'Blaize Talagi', 'Sean Russell',
          'Dylan Brown', 'Mitchell Moses', 'Reagan Campbell-Gillard', 'Joey Lussick', 'Junior Paulo',
          'Shaun Lane', 'Bryce Cartwright', 'J\'maine Hopgood']

team_2_name  = "Sea Eagles"
team_2 = ['Tom Trbojevic', 'Raymond Tuaimalo Vaega', 'Tolutau Koula', 'Reuben Garrick',
          'Jaxson Paulo', 'Luke Brooks', 'Daly Cherry-Evans', 'Taniela Paseka', 'Lachlan Croker', 'Josh Aloiai',
          'Haumole Olakau\'atu', 'Ben Trbojevic', 'Jake Trobojevic']



names = {1: 'Fullback', 2: 'Winger', 3: 'Centre', 4: 'Centre', 5: 'Winger', 6: 'Five-Eight', 7: 'Halfback',
         8: 'Prop', 9: 'Hooker', 10: 'Prop', 11: '2nd Row', 12: '2nd Row', 13: 'Lock'}




In [ ]:
# Get all the match data 

team_1_attack = filtered_df[filtered_df[f"{team_1_name} Attack"] != -1][f"{team_1_name} Attack"].mean()
team_1_defense = filtered_df[filtered_df[f"{team_1_name} Defense"] != -1][f"{team_1_name} Defense"].mean()

team_2_attack = filtered_df[filtered_df[f"{team_2_name} Attack"] != -1][f"{team_2_name} Attack"].mean()
team_2_defense = filtered_df[filtered_df[f"{team_2_name} Defense"] != -1][f"{team_2_name} Defense"].mean()


print(f"{team_1_name} Attack: {team_1_attack} Defense: {team_1_defense}\n{team_2_name} Attack: {team_2_attack} Defense: {team_2_defense}")

Eels Attack: 24.458333333333332 Defense: 23.916666666666668
Sea Eagles Attack: 21.347826086956523 Defense: 22.91304347826087


In [ ]:
def find_player_data(p):
    tries = (p[p['Tries']!= "-"])['Tries'].count()

    tackle_efficiency = p['Tackle Efficiency']
    tackle_efficiency = tackle_efficiency.replace('-', '1.0')
    tackle_efficiency= tackle_efficiency.str.rstrip('%').astype(float) / 100

    
    games = p['Tries'].count() 
    
    try_per_games = tries/games 
    
    return tackle_efficiency.mean(), try_per_games

In [ ]:
# Find the top try scorers for each team 

team_1_tef = {}
team_1_tpg = {}

team_2_tef = {}
team_2_tpg = {}

for player_1, player_2, position in zip(team_1, team_2, names.keys()):
    try:
        
        try:
            p_1 = players[player_1]
            tackle_efficiency, try_per_game = find_player_data(p_1)
        except:
             tackle_efficiency, try_per_game = -1, -1
            

        team_1_tef[position] = tackle_efficiency
        team_1_tpg[position] = try_per_game
        
        
        try:
            p_2 = players[player_2]
            tackle_efficiency_2, try_per_game_2 = find_player_data(p_2)
        except:
             tackle_efficiency_2, try_per_game_2  = -1 -1

        team_2_tef[position] = tackle_efficiency_2
        team_2_tpg[position] = try_per_game_2
        
        
        
        print(f"Pos: {position}\tName: {player_1}\tTEF: {tackle_efficiency:.3f}\tTPG: {try_per_game:.3f}")
        print(f"Pos: {position}\tName: {player_2}\tTEF: {tackle_efficiency_2:.3f}\tTPG: {try_per_game_2:.3f}\n")
        
    except Exception as ex:
        print(f'{player_1} not found {ex}')
        p = None 



Pos: 1	Name: Clinton Gutherson	TEF: 0.736	TPG: 0.522
Pos: 1	Name: Tom Trbojevic	TEF: 0.703	TPG: 0.545

Pos: 2	Name: Morgan Harper	TEF: 0.798	TPG: 0.000
Pos: 2	Name: Raymond Tuaimalo Vaega	TEF: 0.708	TPG: 0.167

Pos: 3	Name: Will Penisini	TEF: 0.821	TPG: 0.292
Pos: 3	Name: Tolutau Koula	TEF: 0.723	TPG: 0.222

Pos: 4	Name: Blaize Talagi	TEF: -1.000	TPG: -1.000
Pos: 4	Name: Reuben Garrick	TEF: 0.769	TPG: 0.476

Pos: 5	Name: Sean Russell	TEF: 0.736	TPG: 0.529
Pos: 5	Name: Jaxson Paulo	TEF: 0.646	TPG: 0.333

Pos: 6	Name: Dylan Brown	TEF: 0.879	TPG: 0.176
Pos: 6	Name: Luke Brooks	TEF: 0.742	TPG: 0.188

Pos: 7	Name: Mitchell Moses	TEF: 0.796	TPG: 0.263
Pos: 7	Name: Daly Cherry-Evans	TEF: 0.904	TPG: 0.238

Pos: 8	Name: Reagan Campbell-Gillard	TEF: 0.852	TPG: 0.214
Pos: 8	Name: Taniela Paseka	TEF: 0.911	TPG: 0.000

Pos: 9	Name: Joey Lussick	TEF: 0.859	TPG: 0.000
Pos: 9	Name: Lachlan Croker	TEF: 0.892	TPG: 0.273

Pos: 10	Name: Junior Paulo	TEF: 0.880	TPG: 0.150
Pos: 10	Name: Josh Aloiai	TEF: 0.9

In [ ]:
team_2_tpg

{1: 0.5454545454545454,
 2: 0.16666666666666666,
 3: 0.2222222222222222,
 4: 0.47619047619047616,
 5: 0.3333333333333333,
 6: 0.1875,
 7: 0.23809523809523808,
 8: 0.0,
 9: 0.2727272727272727,
 10: 0.0625,
 11: 0.34782608695652173,
 12: 0.18181818181818182}

In [ ]:
avg_tef_1 = np.array(list(team_1_tef.values())).mean() 
avg_tpg_1 = np.array(list(team_1_tpg.values()))
avg_tpg_1 = np.median(avg_tpg_1)

avg_tef_2 = np.array(list(team_2_tef.values())).mean() 
avg_tpg_2 = np.array(list(team_2_tpg.values()))
avg_tpg_2 = np.median(avg_tpg_2)


print(f"{team_1_name} TEF: {avg_tef_1:.2f} TPG: {avg_tpg_1:.2f}")
print(f"{team_2_name} TEF: {avg_tef_2:.2f} TPG: {avg_tpg_2:.2f}")

Eels TEF: 0.70 TPG: 0.18
Sea Eagles TEF: 0.80 TPG: 0.23


In [ ]:
# Compare Wingers

winger_left = 2
centre_left = 3

winger_right = 5
centre_right = 4


winger_left_1_tef = team_1_tef[winger_left]
winger_left_2_tef = team_2_tef[winger_left]

centre_left_1_tef = team_1_tef[centre_left]
centre_left_2_tef = team_2_tef[centre_left]


winger_right_1_tef = team_1_tef[winger_right]
winger_right_2_tef = team_2_tef[winger_right]

centre_right_1_tef = team_1_tef[centre_right]
centre_right_2_tef = team_2_tef[centre_right]


if winger_left_1_tef != -1 and centre_left_1_tef != -1:
    # Calculate the average
    left_tef_1 = (winger_left_1_tef + centre_left_1_tef) / 2
else:
    # Take the non-negative value
    left_tef_1 = winger_left_1_tef if winger_left_1_tef != -1 else centre_left_1_tef


if winger_right_1_tef != -1 and centre_right_1_tef != -1:
    # Calculate the average
    right_tef_1 = (winger_right_1_tef + centre_right_1_tef) / 2
else:
    # Take the non-negative value
    right_tef_1 = winger_right_1_tef if winger_right_1_tef != -1 else centre_right_1_tef


if winger_left_2_tef != -1 and centre_left_2_tef != -1:
    # Calculate the average
    left_tef_2 = (winger_left_2_tef + centre_left_2_tef) / 2
else:
    # Take the non-negative value
    left_tef_2 = winger_left_2_tef if winger_left_2_tef != -1 else centre_left_2_tef


if winger_right_2_tef != -1 and centre_right_2_tef != -1:
    # Calculate the average
    right_tef_2 = (winger_right_2_tef + centre_right_2_tef) / 2
else:
    # Take the non-negative value
    right_tef_2 = winger_right_2_tef if winger_right_2_tef != -1 else centre_right_2_tef






# print(f"{winger_left_1_tef}\t{winger_left_2_tef}\t{winger_right_1_tef}\t{winger_right_2_tef}")

# Teams Attack * OP Teams Defense * (teams Avg TPG) * (op teams avg def) * TPG * [OP TEF Winger + Center Average] 
home_bonus = 1.05



win_def_1 = (team_1_attack + team_2_defense)/2

win_def_2 = (team_2_attack + team_1_defense)/2

# Normalise against each other
max_win_def = win_def_1 if win_def_1 > win_def_2 else win_def_2
win_def_1 = win_def_1/max_win_def
win_def_2 = win_def_2/max_win_def


w1 = 0.2
w2 = 0.075
w3 = 0.4
w4 = 0.25
w5 = 0.075

w1_l = (win_def_1*w1 + (avg_tpg_1)*w2 + (team_1_tpg[winger_left])*w3  + (1-left_tef_2)*w4 + (1-avg_tef_2)*w5)*home_bonus
w2_l = win_def_2*w1 + (avg_tpg_2)*w2 + (team_2_tpg[winger_left])*w3  + (1-left_tef_1)*w4+ (1-avg_tef_1)*w5

w1_r = (win_def_1*w1 + (avg_tpg_1)*w2 + (team_1_tpg[winger_right])*w3  + (1-right_tef_2)*w4 + (1-avg_tef_2)*w5)*home_bonus
w2_r = win_def_2*w1 + (avg_tpg_2)*w2 + (team_2_tpg[winger_right])*w3  + (1-right_tef_1)*w4+ (1-avg_tef_1)*w5


w1 = 0.2
w2 = 0.1
w3 = 0.5
w4 = 0.1

w1_fb = (win_def_1*w1 + (avg_tpg_1)*w2 + (team_1_tpg[1])*w3 + (1-avg_tef_2)*w4)*home_bonus
w2_fb = win_def_2*w1 + (avg_tpg_2)*w2 + (team_2_tpg[1])*w3  + (1-avg_tef_1)*w4



# Define the width for each field
width = 10

# Print the header
print(f"{'Team Name':<{width}} {'Attack':>{width}} {'Defense':>{width}} {'TPG':>{width}} {'TEF':>{width}} {'LTEF':>{width}} {'RTEF':>{width}} {'LW APG':>{width}} {'RW APG':>{width}} {'FB APG':>{width}}")

# Print the formatted strings with specified widths for the results


print(f"{team_1_name:<{width}} {team_1_attack:>{width}.2f} {team_1_defense:>{width}.2f} {avg_tpg_1:>{width}.2f} {avg_tef_1:>{width}.2f} {left_tef_1:>{width}.2f} {right_tef_1:>{width}.2f} {team_1_tpg[winger_left]:>{width}.2f} {team_1_tpg[winger_right]:>{width}.2f} {team_1_tpg[1]:>{width}.2f}")
print(f"{team_2_name:<{width}} {team_2_attack:>{width}.2f} {team_2_defense:>{width}.2f} {avg_tpg_2:>{width}.2f} {avg_tef_2:>{width}.2f} {left_tef_2:>{width}.2f} {right_tef_2:>{width}.2f} {team_2_tpg[winger_left]:>{width}.2f} {team_2_tpg[winger_right]:>{width}.2f} {team_2_tpg[1]:>{width}.2f}")
print('\n')

print(f"{team_1_name:<{width}} {team_1[winger_left-1]}: {w1_l:.3f}\n{team_2_name:<{width}} {team_2[winger_left-1]}: {w2_l:.3f}")
print(f"{team_1_name:<{width}} {team_1[winger_right-1]}: {w1_r:.3f}\n{team_2_name:<{width}} {team_2[winger_right-1]}: {w2_r:.3f}")

print('\n')
print(f"{team_1_name:<{width}} {team_1[1-1]}: {w1_fb:.3f}\n{team_2_name:<{width}} {team_2[1-1]}: {w2_fb:.3f}")


Team Name      Attack    Defense        TPG        TEF       LTEF       RTEF     LW APG     RW APG     FB APG
Eels            24.46      23.92       0.18       0.70       0.81       0.74       0.00       0.53       0.52
Sea Eagles      21.35      22.91       0.23       0.80       0.72       0.71       0.17       0.33       0.55


Eels       Morgan Harper: 0.315
Sea Eagles Raymond Tuaimalo Vaega: 0.345
Eels       Sean Russell: 0.539
Sea Eagles Jaxson Paulo: 0.430


Eels       Clinton Gutherson: 0.524
Sea Eagles Tom Trbojevic: 0.517


In [ ]:
print(f"Home: {win_def_1:.2f} Away: {win_def_2:.2f}")

Home: 1.00 Away: 0.96
